<a href="https://colab.research.google.com/github/XescLlabres/tfgFLM/blob/main/HUSE_per_pacient/RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data1 = pd.read_csv('df_gower.csv')

In [ ]:
data1.drop(['ID', 'CLAVE'], axis=1)

,RECIST,GENDER,AGE_AT_TACE,DAYS_CT_PRETACE,WEIGHT,HEIGHT,BMI,BMI_CATEGORY,HCV,Alcohol _abuse\n,...,Tumoral_Distribution,I,II,III,IVa,IVb,V,VI,VII,VIII
0,0.0,1.0,82.0,47.0,64.0,1.47,29.62,3.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,1.0,71.0,26.0,55.0,1.52,23.81,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,48.0,75.0,65.0,1.69,22.76,2.0,1.0,1.0,...,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
3,1.0,0.0,77.0,110.0,68.0,1.59,26.90,3.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,54.0,28.0,65.0,1.70,22.49,2.0,1.0,0.0,...,2.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,1.0,1.0,83.0,194.0,75.0,1.60,29.30,3.0,1.0,0.0,...,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
127,1.0,0.0,67.0,578.0,94.0,1.74,31.05,4.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
128,1.0,0.0,67.0,52.0,65.0,1.74,21.47,2.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
129,1.0,0.0,74.0,269.0,83.0,1.65,30.49,4.0,0.0,1.0,...,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE


data1.columns = data1.columns.str.strip()

y = data1.loc[:, "RECIST"].values

X = data1.loc[:, (data1.columns != "RECIST") & (data1.columns != "ID") & (data1.columns != "CLAVE")].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
smote = SMOTE(random_state = 42)
X_smote, y_smote= smote.fit_resample(X_scaled, y)

# Definir la graella d'hiperparàmetres
param_grid = {
    'n_estimators': [5, 10, 20],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced', None]
}

# Configurar el model i el Grid Search
rf_model = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    scoring='roc_auc',  # Optimitzem segons l'Àrea Sota la Corba (ROC AUC)
    cv=5,               # Validació creuada amb 5 particions
    verbose=2,
    n_jobs=-1           # Utilitza tots els nuclis disponibles
)

# Entrenar el Grid Search
grid_search.fit(X_smote, y_smote)

# Obtenir els millors hiperparàmetres
best_params = grid_search.best_params_
print("Millors hiperparàmetres:", best_params)

# Entrenar el model final amb els millors hiperparàmetres
best_rf_model = grid_search.best_estimator_


Fitting 5 folds for each of 216 candidates, totalling 1080 fits


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


Millors hiperparàmetres: {'class_weight': 'balanced', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 20}


In [ ]:
#Millors hiperparàmetres: {'class_weight': 'balanced', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut
from imblearn.over_sampling import SMOTE

data1.columns = data1.columns.str.strip()

y = data1.loc[:, "RECIST"].values

X = data1.loc[:, (data1.columns != "RECIST") & (data1.columns != "ID") & (data1.columns != "CLAVE")].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Crear el model de classificació i l'objecte LOOCV
loo = LeaveOneOut()
model = RandomForestClassifier(
    n_estimators=200,  # Nombre d'arbres
    max_depth= None,
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=42,   # Per resultats consistents
    class_weight= 'balanced'  # Manejar classes desequilibrades
)

# Inicialitzar arrays per emmagatzemar resultats
all_bootstrap_probabilities = []  # Per emmagatzemar les 1000 probabilitats de cada pacient

# Aplicar LOOCV amb Bootstrap
for train_index, test_index in loo.split(X_scaled):
    # Separar dades de training i test
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Aplicar SMOTE per augmentar les dades
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

    # Probabilitats bootstrap per al pacient deixat fora
    bootstrapped_probabilities = []

    # Generar 1000 mostres bootstrap i fer prediccions
    for _ in range(100):
        # Generar mostra bootstrap
        X_train_bootstrap, y_train_bootstrap = resample(X_train_smote, y_train_smote, replace=True)

        # Entrenar el model amb la mostra bootstrap
        model.fit(X_train_bootstrap, y_train_bootstrap)

        # Predir la probabilitat de classe 1 per al pacient deixat fora
        prob = model.predict_proba(X_test)[:, 1][0]  # Probabilitat de classe 1
        bootstrapped_probabilities.append(prob)

    # Emmagatzemar les probabilitats bootstrap per aquest pacient
    all_bootstrap_probabilities.append(bootstrapped_probabilities)

# Convertir la llista de probabilitats a un DataFrame de pandas
df_probabilities = pd.DataFrame(all_bootstrap_probabilities)

print(df_probabilities.head())

df_probabilities.to_csv("random_forest_bootstrap_probabilities.csv", index=False)


       0      1      2      3      4      5      6      7      8      9
0  0.705  0.570  0.525  0.690  0.625  0.695  0.685  0.680  0.675  0.665
1  0.770  0.860  0.715  0.740  0.805  0.770  0.745  0.780  0.725  0.785
2  0.695  0.600  0.660  0.595  0.625  0.740  0.780  0.630  0.660  0.630
3  0.735  0.605  0.750  0.725  0.630  0.665  0.685  0.770  0.720  0.775
4  0.825  0.900  0.885  0.845  0.850  0.830  0.835  0.775  0.845  0.805


In [ ]:

df_probabilities = pd.DataFrame(all_bootstrap_probabilities)

data1 = pd.read_csv('df_gower.csv')
data1.drop(['CLAVE'],axis=1,inplace=True)
df_probabilities.insert(0, 'ID', data1['ID'].values)
df_probabilities.to_csv("huse4bootstrap_probabilities_id.csv", index=False)



print("Archivo guardado con el identificador 'id' en la primera columna.")

Archivo guardado con el identificador 'id' en la primera columna.


In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

In [ ]:
taula1 = pd.read_csv('huse4bootstrap_probabilities_id.csv')
taula2 = pd.read_csv('df_gower.csv')

taula2.drop(['CLAVE'],axis=1,inplace=True)

In [ ]:

id_column = taula1.iloc[:, 0]

probabilities = taula1.iloc[:, 1:]

y_pred = (probabilities.mean(axis=1) >= 0.5).astype(int)

taula1_processed = pd.DataFrame({'ID': id_column, 'RECIST': y_pred})

taula2_processed = taula2[['ID', 'RECIST']]
taula2_processed.rename(columns={'RECIST': 'y_true'}, inplace=True)

taula05 = pd.merge(taula1_processed, taula2_processed, on='ID')

print(taula05.head())

      ID  RECIST  y_true
0  222.0       1     0.0
1  346.0       1     0.0
2  613.0       1     1.0
3  697.0       1     1.0
4  745.0       1     0.0


<ipython-input-10-735a19bb8885>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taula2_processed.rename(columns={'RECIST': 'y_true'}, inplace=True)


In [ ]:
y_pred = taula05['RECIST']
y_true = taula05['y_true']

cm = confusion_matrix(y_true, y_pred)
print("Matriu de confusió:")
print(cm)

precision = precision_score(y_true, y_pred)
print(f"Precisión: {precision:.2f}")

recall = recall_score(y_true, y_pred)
print(f"Recall: {recall:.2f}")

f1 = f1_score(y_true, y_pred)
print(f"F1-Score: {f1:.2f}")

accuracy = accuracy_score(y_true, y_pred)
print(f"Exactitut: {accuracy:.2f}")

Matriz de Confusión:
[[  0  21]
 [  3 107]]
Precisión: 0.84
Recall: 0.97
F1-Score: 0.90
Exactitud: 0.82


In [ ]:
df_data = pd.read_csv("df_gower.csv")
df_data.drop(['CLAVE'],axis=1,inplace=True)

df_probabilities = pd.read_csv("huse4bootstrap_probabilities_id.csv")

df_data_subset = df_data[['ID', 'RECIST']]

if len(df_data_subset) == len(df_probabilities):
    df_final = pd.concat([df_data_subset, df_probabilities], axis=1)

    df_final.to_csv("final_table.csv", index=False)

    print("Tabla final creada y guardada en 'final_table.csv'.")
else:
    print("Error: Los DataFrames tienen diferente número de filas. Verifica tus datos.")

Tabla final creada y guardada en 'final_table.csv'.


In [ ]:
df = pd.read_csv('final_table.csv')

id_column = 'ID'
diagnosis_column = 'RECIST'
prob_columns = df.columns[3:]
def calcular_intervalo(probabilidades, li, ls):
    lower_bound = np.percentile(probabilidades, li)
    upper_bound = np.percentile(probabilidades, ls)
    return lower_bound, upper_bound

In [ ]:
intervalos_confianza = []

for index, row in df.iterrows():
    probabilidades = row[prob_columns].values
    lower, upper = calcular_intervalo(probabilidades, 2.5, 97.5)
    intervalos_confianza.append([row[id_column], row[diagnosis_column], lower, upper])

# Crear un nuevo DataFrame con los intervalos de confianza
df_intervalos = pd.DataFrame(intervalos_confianza, columns=[id_column, diagnosis_column, 'lower_bound', 'upper_bound'])

In [ ]:
def calcular_TP(a, b):
    TP = min(1 - a, b)
    return TP
def calcular_AP(a,b):
    AP = min(a, 1-b)
    return AP
def calcular_EP(a,b):
    EP = (b-a)
    return EP

In [ ]:
# Añadir la columna TP aplicando la función calcular_TP a lower_bound y upper_bound
df_intervalos['TP'] = df_intervalos.apply(lambda row: calcular_TP(row['lower_bound'], row['upper_bound']), axis=1)

# Añadir la columna AP aplicando la función calcular_AP a lower_bound y upper_bound
df_intervalos['AP'] = df_intervalos.apply(lambda row: calcular_AP(row['lower_bound'], row['upper_bound']), axis=1)

# Añadir la columna EP aplicando la función calcular_EP a lower_bound y upper_bound
df_intervalos['EP'] = df_intervalos.apply(lambda row: calcular_EP(row['lower_bound'], row['upper_bound']), axis=1)

In [ ]:
# Calcular medias de TP, AP y EP
medias = df_intervalos[['TP', 'AP', 'EP']].mean()
print("Medias:")
print(medias)

Medias:
TP    0.369169
AP    0.207396
EP    0.161773
dtype: float64
